<a href="https://colab.research.google.com/github/dman82499/CSE-3521-Irony-Detection/blob/main/HW02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from collections import Counter

# Import training data and store as DF
positive_train_URL = 'https://raw.githubusercontent.com/jeniyat/CSE-5521-SP21/master/HW/HW2/Data/train/Positive.txt'
negative_train_URL = 'https://raw.githubusercontent.com/jeniyat/CSE-5521-SP21/master/HW/HW2/Data/train/Negative.txt'
neutral_train_URL = 'https://raw.githubusercontent.com/jeniyat/CSE-5521-SP21/master/HW/HW2/Data/train/Neutral.txt'

positive_train_DF = pd.read_table(positive_train_URL, header=None);
negative_train_DF = pd.read_table(negative_train_URL, header=None);
neutral_train_DF = pd.read_table(neutral_train_URL, header=None);

# Import testing data and store as DF
positive_test_URL = 'https://raw.githubusercontent.com/jeniyat/CSE-5521-SP21/master/HW/HW2/Data/test/Positive.txt'
negative_test_URL = 'https://raw.githubusercontent.com/jeniyat/CSE-5521-SP21/master/HW/HW2/Data/test/Negative.txt'
neutral_test_URL = 'https://raw.githubusercontent.com/jeniyat/CSE-5521-SP21/master/HW/HW2/Data/test/Neutral.txt'

positive_test_DF = pd.read_table(positive_test_URL, header=None);
negative_test_DF = pd.read_table(negative_test_URL, header=None);
neutral_test_DF = pd.read_table(neutral_test_URL, header=None);

# Converts DF into list of strings containing Tweets
positive_train = positive_train_DF[0].tolist()
negative_train = negative_train_DF[0].tolist()
neutral_train = neutral_train_DF[0].tolist()
positive_test = positive_test_DF[0].tolist()
negative_test = negative_test_DF[0].tolist()
neutral_test = neutral_test_DF[0].tolist()

num_of_pos_doc = len(positive_train)
num_of_neg_doc = len(negative_train)
num_of_neu_doc = len(neutral_train)
total_num_doc = num_of_neg_doc + num_of_neu_doc + num_of_pos_doc

prob_pos = num_of_pos_doc / total_num_doc
prob_neg = num_of_neg_doc / total_num_doc
prob_neu = num_of_neu_doc / total_num_doc

vocab = Counter();
vocab_pos = Counter();
vocab_neg = Counter();
vocab_neu = Counter();

total_pos_word = 0;
total_neg_word = 0;
total_neu_word = 0;

# Split positive training data into bag of words
for sent in positive_train:
    word_list = sent.split()

    for word in word_list:
        vocab[word] += 1
        vocab_pos[word] += 1
        total_pos_word += 1

# Split negative training data into bag of words
for sent in negative_train:
    word_list = sent.split()

    for word in word_list:
        vocab[word] += 1
        vocab_neg[word] += 1
        total_neg_word += 1

# Split neutral training data into bag of words
for sent in neutral_train:
    word_list = sent.split()

    for word in word_list:
        vocab[word] += 1
        vocab_neu[word] += 1
        total_neu_word += 1

vocab_size = len(vocab)

# From testing, alpha = 2 yielded the most accurate results at 0.87613 rounded
alpha = 2

# Find probability for positive training data
def find_pos_prob(list_of_words):
    pos_prob_list_of_words = prob_pos

    for word in list_of_words:
        if word in vocab_pos:
            numerator = vocab_pos[word] + alpha
        else:
            numerator = alpha

        denominator = total_pos_word + alpha * vocab_size
        word_prob = numerator / denominator

        pos_prob_list_of_words = pos_prob_list_of_words * word_prob

    return pos_prob_list_of_words

# Find probability for negative training data
def find_neg_prob(list_of_words):
    neg_prob_list_of_words = prob_neg

    for word in list_of_words:
        if word in vocab_neg:
            numerator = vocab_neg[word] + alpha
        else:
            numerator = alpha

        denominator = total_neg_word + alpha * vocab_size
        word_prob = numerator / denominator

        neg_prob_list_of_words = neg_prob_list_of_words * word_prob

    return neg_prob_list_of_words

# Find probability for neutral training data
def find_neu_prob(list_of_words):
    neu_prob_list_of_words = prob_neu

    for word in list_of_words:
        if word in vocab_neu:
            numerator = vocab_neu[word] + alpha
        else:
            numerator = alpha

        denominator = total_neu_word + alpha * vocab_size
        word_prob = numerator / denominator

        neu_prob_list_of_words = neu_prob_list_of_words * word_prob

    return neu_prob_list_of_words

predictions = []

# Positive testing data
for sent in positive_test:
    word_list = sent.split()
    pos_prob_sent = find_pos_prob(word_list)
    neg_prob_sent = find_neg_prob(word_list)
    neu_prob_sent = find_neu_prob(word_list)

    true_sentiment = "+ve"

    max_prob = max(pos_prob_sent, neg_prob_sent, neu_prob_sent)

    if max_prob == pos_prob_sent:
        pred_sentiment = "+ve"
    if max_prob == neg_prob_sent:
        pred_sentiment = "-ve"
    if max_prob == neu_prob_sent:
        pred_sentiment = "0ve"

    predictions.append((sent, true_sentiment, pred_sentiment))

# Negative testing data
for sent in negative_test:
    word_list = sent.split()
    pos_prob_sent = find_pos_prob(word_list)
    neg_prob_sent = find_neg_prob(word_list)
    neu_prob_sent = find_neu_prob(word_list)

    true_sentiment = "-ve"

    max_prob = max(pos_prob_sent, neg_prob_sent, neu_prob_sent)

    if max_prob == pos_prob_sent:
        pred_sentiment = "+ve"
    if max_prob == neg_prob_sent:
        pred_sentiment = "-ve"
    if max_prob == neu_prob_sent:
        pred_sentiment = "0ve"

    predictions.append((sent, true_sentiment, pred_sentiment))


# Neutral testing data
for sent in neutral_test:
    word_list = sent.split()
    pos_prob_sent = find_pos_prob(word_list)
    neg_prob_sent = find_neg_prob(word_list)
    neu_prob_sent = find_neu_prob(word_list)

    true_sentiment = "0ve"

    max_prob = max(pos_prob_sent, neg_prob_sent, neu_prob_sent)

    if max_prob == pos_prob_sent:
        pred_sentiment = "+ve"
    if max_prob == neg_prob_sent:
        pred_sentiment = "-ve"
    if max_prob == neu_prob_sent:
        pred_sentiment = "0ve"

    predictions.append((sent, true_sentiment, pred_sentiment))

correct_pred = 0
total_test_case = 0

for pred in predictions:
    (sent, true_sentiment, pred_sentiment) = pred
    if true_sentiment == pred_sentiment:
        correct_pred += 1
    total_test_case += 1

accuracy = correct_pred / total_test_case
print("Accuracy of ", accuracy)
print("Alpha is ", alpha)


Accuracy of  0.8761290322580645
Alpha is  2
